In [ ]:
# load import

import tensorflow as tf
from pylab import *
from pandas import *
import keras as K
import keras.backend as KB
import os, sys, time, re
from imp import reload
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')


# tf.reset_default_graph()
print(tf.__version__, sys.version)


In [ ]:
INITIAL_LEARNING_RATE
NUM_STEPS_PER_DECAY

In [ ]:
# testing my fjnn wrappers
####################################################################

%matplotlib nbagg
from imp import reload
import tensorflow as tf
from pylab import *
from pandas import *
from statsmodels.api import OLS
import convolutions as conv; reload(conv)
import fjnn
from fjnn._defaults import *
import fjnn.units as fjnu; reload(fjnu)
import fjnn.core as fjnc; reload(fjnc)
import time, shutil, os, sys, re
tf.reset_default_graph()

batch_size = 1256
n_epochs = 1000
train_dir = '/home/ubuntu/deep-learning/data/test_fjnn/'
for f in os.listdir(train_dir):
    os.remove(train_dir+f)
    
w = randn(300)
def generate_data(X):
    return np.reshape( X, (-1, 300) ) @ w

# -validation data --
X_val = randn( 2000, 10, 30 )
Y_val = generate_data(X_val)

# -- deep learning model  --
g = tf.Graph().as_default()

def model(x, scope='model', reuse=False):
    with tf.variable_scope(scope, reuse=reuse) as scope:
        dense1 = fjnu.dense('dense1', x, nout=5, bias=False, W_l2=0.0, activation=tf.nn.relu)
        dense2 = fjnu.dense('dense2', dense1, nout=2, bias=False, activation=tf.nn.relu)
        dense3 = fjnu.dense('dense3', dense2, nout=1, bias=False, activation=tf.identity)
        print(dense1, dense2, dense3)
    return dense3, scope

# training model
x_trg = tf.placeholder(dtype=DTYPE, shape=(None, 10, 30), name='x_trg')
y_trg = tf.placeholder(dtype=DTYPE, shape=(None, ), name='y_trg')
yhat_trg, model_scope = model(x_trg)
trg_error = fjnu.mse(yhat_trg, y_trg, name='trg_error')
tf.add_to_collection( LOSSES, trg_error )
trg_loss = tf.add( tf.add_n( tf.get_collection( REGULARIZERS ) ), trg_error, name='trg_loss' )
tf.add_to_collection( LOSSES, trg_loss )

# validation model
x_val = tf.placeholder(dtype=DTYPE, shape=(None, 10, 30), name='x_val')
y_val = tf.placeholder(dtype=DTYPE, shape=(None, ), name='y_val')
yhat_val, model_scope = model(x_val, scope=model_scope, reuse=True)
val_error = fjnu.mse(yhat_val, y_val, name='val_error')
tf.add_to_collection( LOSSES, val_error)

# summarize shit
fjnc.summarize_tensors( tf.trainable_variables(), summary='variable' )
fjnc.summarize_tensors( tf.get_collection(ACTIVATIONS), summary='activation' )
fjnc.summarize_tensor( tf.global_norm(tf.trainable_variables(), name='trainable_variables'),  summary='training' )
# create the loss summarizer
loss_ema_op = fjnc.summarize_losses()

#optimization varaibles
global_step = tf.contrib.framework.create_global_step()
# optimizer with decaying learning rate
lr = tf.train.exponential_decay( INITIAL_LEARNING_RATE, global_step, NUM_STEPS_PER_DECAY, LEARNING_RATE_DECAY_FACTOR, staircase=True, name='learning_rate' )
fjnc.summarize_tensor(lr, 'training')
opt = tf.train.AdamOptimizer(0.1)
# create a variable ema op
variable_ema_op = fjnc.ema_variables( global_step )
# create the descent op
with tf.control_dependencies([loss_ema_op, variable_ema_op]):
    train_op = fjnc.minimize(opt, trg_loss, global_step=global_step, )

# start the optimization
saver = tf.train.Saver( tf.all_variables() )
summary_op = tf.merge_all_summaries( )
with tf.Session() as sess:
    sess.run( tf.initialize_all_variables() )
    summary_writer = tf.train.SummaryWriter( train_dir, sess.graph )
    # mini=batch sgd
    for n in range(n_epochs):
        X_trg = randn( batch_size, 10, 30 )
        Y_trg = generate_data( X_trg )
        feed_dict = { x_trg: X_trg, y_trg: Y_trg, x_val: X_val, y_val: Y_val }
        _ = sess.run( train_op, feed_dict=feed_dict )
        if n % 10 == 0:
            summary_writer.add_summary( summary_op.eval( feed_dict=feed_dict ), global_step.eval() )
        if (n*10) / n_epochs == (n*10) // n_epochs:
            [trg_loss_value, trg_err_value, val_err_value] = sess.run( [trg_loss, trg_error, val_error], feed_dict=feed_dict)
            print( n, trg_loss_value, trg_err_value, val_err_value )
    Yhat_val = yhat_val.eval( feed_dict=feed_dict )
    

%matplotlib nbagg
plot( Yhat_val, Y_val, '.')






        





In [ ]:
?os.remove

In [ ]:
tf.get_collection(WEIGHTS)[0].name

In [ ]:
set(tf.get_collection(WEIGHTS))

In [ ]:
reload(fjnu)
with tf.Graph().as_default() as g:
    with tf.Session() as sess:
        w = tf.Variable( tf.random_normal((5, 3)) )
        print( tf.nn.l2_loss( tf.matmul(tf.transpose(w), w) ) )
        
        

In [ ]:
# Testing linear deconvoluation with tensorflow
#####################################################
%matplotlib nbagg
from imp import reload
import tensorflow as tf
from pylab import *
from pandas import *
from statsmodels.api import OLS
import convolutions as conv; reload(conv)
import fjnn
import fjnn.units as fjnu; reload(fjnu)
import fjnn.core; reload(fjnn.core)
import time, shutil, os, sys

N = 5000;
P = 5;
H = 100;
dtype = tf.float32;
df, conv_kernels = conv.get_data(N, P, H)
X = conv.build_hankel_tensors(df, P, H)
Y = df['yl'].values
train_dir = '/home/ubuntu/deep-learning/data/linear_deconv_tf'
shutil.rmtree(train_dir)
max_steps = 5000;
 
with tf.Graph().as_default():
    #the training set
    x = tf.placeholder(shape=(None, P, H), dtype=dtype, name='trg_input')
    y = tf.placeholder(shape=(None, ), dtype=dtype)
    with tf.variable_scope('linear_deconv') as model:        
        yhat = fjnu.dense(x, num_output=1, bias=False, W_l2=0.1)
        trg_loss = fjnu.mse(y, yhat, name='trg_loss')        
    tf.add_to_collection('trg_losses', trg_loss)
    
    # the validation set
    x_val = tf.placeholder(shape=(None, P, H), dtype=dtype, name='val_input')
    y_val = tf.placeholder(shape=(None, ), dtype=dtype)
    with tf.variable_scope(model, reuse=True):
        yhat_val = fjnn.units.dense(x_val, num_output=1, bias=False)
    val_loss = fjnu.mse(y_val, yhat_val, name='val_loss')        
    tf.add_to_collection('val_losses', val_loss)
        
    print('trainable vars', [(v.name, v.get_shape()) for v in tf.trainable_variables() ])
    print('training losses', [(v.name, v.get_shape()) for v in tf.get_collection('trg_losses') ])
    print('validation losses', [(v.name, v.get_shape()) for v in tf.get_collection('val_losses') ])
    print( 'yhat shapes', y_val.get_shape(), yhat_val.get_shape(), ((y_val - tf.squeeze(yhat_val)) **2).get_shape() )
    cost_ema_op = fjnn.core.summarize_costs()
    
    #Train the model
    global_step = tf.Variable(0, trainable=False)
    var_ema_op = fjnn.core.ema_variables(global_step)
    opt = tf.train.AdamOptimizer(learning_rate=0.1, epsilon=)        
    with tf.control_dependencies([var_ema_op, cost_ema_op]):
        train_ops = fjnn.core.build_train_ops(opt, global_step, validation=True)

    # set up the various summarizers and variable emas
    fjnn.core.summarize_collection(tf.trainable_variables())
    fjnn.core.summarize_collection(tf.get_collection('activations'))
    fjnn.core.summarize_collection(tf.get_collection('gradients'), 'gradient')        
    print(tf.get_collection('gradients'))

    # Create a saver.
    saver = tf.train.Saver( tf.all_variables() )
    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.merge_all_summaries()

    with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:        
        sess.run(tf.initialize_all_variables())
        print(sess.run(trg_loss, feed_dict={x:X, y:Y}))
        print(sess.run(val_loss, feed_dict={x_val:X, y_val:Y}))          
        # Logging
        summary_writer = tf.train.SummaryWriter(train_dir, sess.graph)
        for step in range(max_steps):
            start_time = time.time()
            _, total_trg_cost, total_val_cost = sess.run(train_ops, feed_dict={x:X, y:Y, x_val:X, y_val:Y})                
            if step % 100 == 0:
                summary_str = sess.run(summary_op, feed_dict={x:X, y:Y, x_val:X, y_val:Y})
                summary_writer.add_summary(summary_str, global_step.eval())
                w = tf.trainable_variables()[0].eval()
                print(global_step.eval(), total_trg_cost, total_val_cost, mean((np.squeeze((np.reshape(X, [-1, P*H])@w)) - df['yl'].values)**2) )
        
ols = OLS(
    endog=df['yl'],
    exog=np.reshape(X, [-1,P*H] )
).fit()
for p in range(P):        
    subplot(221); plot( conv_kernels[p] )
    subplot(222); plot( df['x_%d'%p] )
subplot(223); plot( df['ynl'], '-k', alpha=0.5 )
subplot(224)
plot(ols.params.values, label='ols')

subplot(224); plot(w, label='tf')
gca().legend()
plt.show()

In [ ]:
mean(ols.resid**2)

In [ ]:
# Testing deconvoluation with keras
#####################################################
%matplotlib nbagg

import scipy.linalg as SLA
from numba import jit
import numpy as np;
import os, sys, time
import tensorflow as tf
from pylab import *
from pandas import *
from statsmodels.api import OLS
import convolutions as conv; reload(conv)

N = 5000;
P = 5;
nb_epoch=50;
batch_size=128

H = 100; #the convoluation window length
conv.linear_test(N, P, H, nb_epoch, batch_size)

In [ ]:
%matplotlib nbagg

reload(conv)

df, conv_kernels = conv.get_data(N, P, H)
X = conv.build_hankel_tensors(df, P, H)
ols = OLS(
    endog=df['ynl'],
    exog=np.reshape(X, [-1,P*H] )
).fit()
km = conv.linear_deconv(X, df['ynl'].values, nb_epoch, batch_size)
w = km.get_weights()[0]
for p in range(P):        
    subplot(221); plot( conv_kernels[p] )
    subplot(222); plot( df['x_%d'%p] )
subplot(223); plot( df['ynl'], '-k', alpha=0.5 )
subplot(224)
plot(ols.params.values, label='ols')

subplot(224); plot(w, label='keras')
gca().legend()
plt.show()


In [ ]:
# Testing basic convolution ideas
# Note : np.convolve is a causal convolution
# tf.conv2d is a symmetric inner-product - not a convolution !!!
#######################################################
%matplotlib nbagg

import scipy.linalg as SLA
from numba import jit
import numpy as np;
import os, sys, time
import tensorflow as tf
from pylab import *
from pandas import *

N = 5000;
P = 5;
H = 100;
dtype = tf.float32;
reg_wt = 0.001;
# variables that affect the learning
batch_size = 1024;
max_steps = 5000;
train_dir = '/home/ubuntu/fj/data/basic_conv'
os.makedirs(train_dir, exist_ok=True)

# -- prepare the dataset  --
def gfunc(n, sigma):
    '''gaussian functions'''
    return exp( -pow(linspace(-6*sigma, 6*sigma, n),2)/2/sigma**2 )/sqrt(2*pi*sigma**2)
# the generating convolution kernels 
conv_kernels = [         
    gfunc(H, sigma)*cos(linspace(0,2*pi*(sigma-1), H)) for sigma in range(1,2*P,2)
]
# the input dataframe
df = DataFrame(
    hstack( [
        randn(N, 4), 
        reshape( sin(linspace(0, N/2/pi, N))>0, (N,1))
    ] ),
    columns=['x_%d'%j for j in range(P)]
)
# convolve 'same' is a causal convolution y[t] = \sum_{i=0}^n x[t-i]h[i]
df['y'] = sum(array([ convolve( df['x_%d'%p], conv_kernels[p], 'same') for p in range(P-1) ]), axis=0) + df['x_%d'%(P-1)] + 0.1*randn(N)

if False:
    for p in range(P):
        plot(conv_kernels[p])
    figure();
    plot(df['x_0'], '--')
    plot(df['x_1'], '--')
    plot(df['y'], '-')
    # -- test tensorflow convolution -- it is symmetric innerproduct not a 'convolution' --    
    with tf.Graph().as_default() as g, tf.Session() as sess:
        x = tf.Variable( initial_value=array([0, 1.0, 20.0, 30]), dtype=tf.float32 )
        h = tf.Variable( initial_value=array([0.1, 0.5, 0.2, 0.2]), dtype=tf.float32 )
        xx = tf.reshape(x, (-1, 4,1,1))
        hh = tf.reshape(h, (4,1,1, 1))
        sess.run(x.initializer)
        sess.run(h.initializer)
        tf.initialize_all_tables()
        cc = tf.nn.conv2d(xx, hh, strides=[1,1,1,1], padding='VALID')
        print(tf.squeeze(xx).eval(), '\n', tf.squeeze(hh).eval(),'\n', tf.squeeze(cc).eval())

@jit(nopython=True, nogil=True)
def hankel(x, w):
    '''
    Reshape the x vector into a causal hankel matrix of width w
    op[t] = [x_t ... x_{t-w}]
    '''
    n = len(x)
    op = zeros( (n, w))
    for t in range(n):
        for v in range(w):
            if t - v < 0 : break;
            op[t, v] = x[t-v]
    return op

def tensorize_x(df=df, P=P, w=H):
    ''' convert the input columns into an [N,P,H] tensor '''
    return np.transpose( 
        np.array( [ hankel(df['x_%d'%p].values, w) for p in range(P) ]), 
        [1,2,0] 
    )        

# --- don't use tensorflow convolutions -- instead reshape the data ----
with tf.Graph().as_default() as g , tf.Session() as sess: 
    # placeholders for the input and output variables
    y = tf.placeholder(tf.float32, shape=N, name='y')    
    x = tf.placeholder(tf.float32, shape=(N, H, P), name='x')    
    x_batch, y_batch = tf.train.batch(
        [ tensorize_x(), df['y'].values], 
        batch_size=batch_size,
        num_threads=1,
        enqueue_many=True
    )
    x = tf.cast(x_batch, dtype)
    y = tf.cast(y_batch, dtype)
    print('x shape', x.get_shape())    
#     tf.scalar_summary( 'x/len', x.get_shape()[0].value )
#     tf.histogram_summary( 'x/histogram', x )
#     # define the name scope for the convolution layer
#     with tf.name_scope('conv0') as scope:
#         # create the P convolution kernels (or get it if it exists)         
#         kernel = tf.get_variable(
#             name='kernel', shape=[H,1,P,1], dtype=dtype,
#             initializer=tf.contrib.layers.xavier_initializer_conv2d(uniform=False, dtype=dtype) 
#         )
#         # apply the Hx1 convolution kernel on each of the P input channels separately 
#         # and squeeze out redudant dimensions
#         conv = tf.squeeze( tf.nn.depthwise_conv2d(
#                 input=tf.reshape(x, [-1,H,1,P]), filter=kernel, 
#                 strides=[1, 1, 1, 1], padding='VALID', name='conv', 
#         ), name='squeezed-conv' )
#         # add some summaries
#         tf.histogram_summary(kernel.name + '/activations', kernel)
#         tf.scalar_summary(kernel.name + '/sparsity', tf.nn.zero_fraction(kernel))
#     # -- create the output layer --
#     with tf.name_scope('output') as scope:
#         W = tf.ones( [P, 1])
#         yhat = tf.matmul(conv, W, name='yhat')
#     # -- define the loss function and regularizers --    
#     with tf.name_scope('loss') as scope:
#         l2error = tf.reduce_mean( tf.square(y-yhat), name='l2error')
#         tf.add_to_collection('losses', l2error)
#         # create a "differencing" regularization matrix
#         regmtx = tf.constant(value=SLA.circulant([1,-1]+[0]*(H-2))[1:], dtype=dtype )
#         l2reg = tf.nn.l2_loss( tf.matmul( regmtx, tf.squeeze(kernel) ), name='l2reg') ;
#         tf.add_to_collection('losses', l2reg)
#         total_loss = l2error + l2reg*reg_wt;    
#         # add some summaries
#         tf.scalar_summary( total_loss.op.name, total_loss )
#         tf.scalar_summary( l2reg.op.name, l2reg )
#         tf.scalar_summary( l2error.op.name, l2error )
#     # -- define a training step --
#     train_step = tf.train.AdadeltaOptimizer().minimize( total_loss )            
#     # -- actually run the model --
#     # Create a saver.
#     saver = tf.train.Saver( tf.all_variables() )
#     # Build the summary operation based on the TF collection of Summaries.
#     summary_op = tf.merge_all_summaries()
#     # Initialization operation
#     sess.run(tf.initialize_all_variables())
#     # create a coordinator to manage the threads
#     coord = tf.train.Coordinator()
#     # Start the queue runners (created in tf.train.batch)-> to read the input
#     threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#     # Logging
#     summary_writer = tf.train.SummaryWriter(train_dir, sess.graph)    
#     try:
#         for step in range(max_steps):
#             # has a thread signalled stopping ?
#             if coord.should_stop():
#                 break;
#             train_step.run()
#             start_time = time.time()
#             loss_value = sess.run(total_loss)
#             duration = time.time() - start_time
#             assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
#             if step % 100 == 0:
#                 num_examples_per_step = batch_size
#                 examples_per_sec = num_examples_per_step / duration
#                 sec_per_batch = float(duration)
#                 format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
#                 print (format_str%(datetime.now(), step, loss_value, examples_per_sec, sec_per_batch))
#             if step % 500 == 0:
#                 summary_writer.add_summary(sess.run(summary_op), step)            
#     except tf.errors.OutOfRangeError:
#         print('Done training -- epoch limit reached')
#     finally:
#         # When done, ask the threads to stop.
#         coord.request_stop()
#     # Wait for threads to finish.
#     coord.join(threads)
#     conv_filter = kernel.eval()


In [ ]:
# tensorflow Conv nets (cifar10) tutorial
########################################################
from tensorflow.models.image.cifar10 import cifar10, cifar10_input
import fj_cifar10
reload(fj_cifar10)
# tf.reset_default_graph()
images = tf.placeholder(tf.float32, [fj_cifar10.batch_size,32,32,3])
labels = tf.placeholder(tf.int64, [fj_cifar10.batch_size])


with tf.variable_scope('conv1') as scope:
    # 64-channels of 5x5x3 convolution kernels.
    kernel = fj_cifar10._variable_with_weight_decay('weights', shape=[5, 5, 3, 64], stddev=5e-2, wd=0.0)
    # stride = 1, i.e no downsampling.
    conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
    biases = fj_cifar10._variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
    bias = tf.nn.bias_add(conv, biases)
    # shape is [batch, height, width, 64]
    conv1 = tf.nn.relu(bias, name=scope.name)
    fj_cifar10._activation_summary(conv1)
# -- pool1 : pooling of 3x3 for each channel, with downsampling factor of 2.
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
# -- norm1: local-response normalization --
# sqr_sum[a, b, c, d] =  sum(input[a, b, c, d - depth_radius : d + depth_radius + 1] ** 2)
# output = input / (bias + alpha * sqr_sum) ** beta
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

# -- conv2 --
with tf.variable_scope('conv2') as scope:
    # 64channels of 5x5x64 convolution kernels
    kernel = fj_cifar10._variable_with_weight_decay('weights', shape=[5, 5, 64, 64], stddev=5e-2, wd=0.0)
    # no downsampling. shape=[batch, height, width, 64]
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = fj_cifar10._variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)
    fj_cifar10._activation_summary(conv2)
# -- norm2 --
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
# -- pool2: 3x3 pooling and downsample by 2
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
# -- local3: fully connected --
with tf.variable_scope('local3') as scope:
    # Reshape the tensor - so we can perform a single matrix multiply.
    reshape = tf.reshape(pool2, [fj_cifar10.batch_size, -1])
    dim = reshape.get_shape()[1].value #should be width/2/2 x height/2/2 x 64 
    weights = fj_cifar10._variable_with_weight_decay('weights', shape=[dim, 384], stddev=0.04, wd=0.004)
    biases = fj_cifar10._variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
    local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
    fj_cifar10._activation_summary(local3)
# --  local4 --
with tf.variable_scope('local4') as scope:
    weights = fj_cifar10._variable_with_weight_decay('weights', shape=[384, 192], stddev=0.04, wd=0.004)
    biases = fj_cifar10._variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
    # batch_size x 192
    local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
    fj_cifar10._activation_summary(local4)
# softmax, i.e. softmax(WX + b) -> still in logits. exp is not applied.
with tf.variable_scope('softmax_logits') as scope:
    weights = fj_cifar10._variable_with_weight_decay('weights', [192, fj_cifar10.NUM_CLASSES], stddev=1/192.0, wd=0.0)
    biases = fj_cifar10._variable_on_cpu('biases', [fj_cifar10.NUM_CLASSES], tf.constant_initializer(0.0))
    # shape = batch_size x NUM_CLASSES
    softmax_logits = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
    fj_cifar10._activation_summary(softmax_logits)

    
#converts labels into 1-hot vectors. ent[x,y] = \sum_i y_i logit_i - log(\sum_k logit_k)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(softmax_logits, labels, name='cross_entropy_per_example')
# reduce along training dimension
cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')    
tf.add_to_collection('losses', cross_entropy_mean)

# sess.run(tf.initialize_all_variables())
# for k, v in locals().items():
#     if type(v) is tf.python.framework.ops.Tensor:
#         print (k, v.name, v.get_shape())
g = tf.get_default_graph()
for k in g.get_all_collection_keys():
    for v in g.get_collection(k):
        print (k, v.name, v.get_shape())

In [ ]:
# MNIST conv NN
###############################################################
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.Graph().as_default() as g , tf.Session() as sess: 

    # the input and output placeholders
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])

    # first layer. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels.
    W_conv1 = weight_variable( [5, 5, 1, 32] )
    b_conv1 = bias_variable( [32] )

    #reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of input channels.
    x_image = tf.reshape(x, [-1, 28, 28,1])

    # convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.
    h_conv1 = tf.nn.relu( conv2d( x_image, W_conv1 ) + b_conv1 )
    h_pool1 = max_pool_2x2( h_conv1 )

    #The second layer will have 64 features for each 5x5 patch.
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    # reshape operates with the right most index as the fastest (i.e. it fills by counting from right to left). Therefore, it makes sense to put the sample index along the first index.

    # Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. 
    # We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.
    W_fc1 = weight_variable( [7 * 7 * 64, 1024] )
    b_fc1 = bias_variable( [1024] )

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu( tf.matmul(h_pool2_flat, W_fc1) + b_fc1 )

    # We create a placeholder for the probability that a neuron's output is kept during dropout. 
    # This allows us to turn dropout on during training, and turn it off during testing. 
    # TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.1

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


    # Finally, we add a softmax layer, just like for the one layer softmax regression above.
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.nn.softmax( tf.matmul(h_fc1_drop, W_fc2) + b_fc2 )

    cross_entropy = tf.reduce_mean( -tf.reduce_sum( y_ * tf.log(y_conv), reduction_indices=[1] ) )
    train_step = tf.train.AdamOptimizer(1e-4).minimize( cross_entropy )
    correct_prediction = tf.equal( tf.argmax(y_conv, 1), tf.argmax(y_, 1) )
    accuracy = tf.reduce_mean( tf.cast(correct_prediction, tf.float32) )
    sess.run( tf.initialize_all_variables() )
    

    trg_acc = [];
    tst_acc = [];
    idx = [];
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval( feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0} )
            test_accuracy = accuracy.eval( feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0} )
            print("%d | training=%g, testing=%g"%(i, train_accuracy, test_accuracy))
            trg_acc.append(train_accuracy)
            tst_acc.append(test_accuracy)
            idx.append(i)
        train_step.run( feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5} )

    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    %matplotlib nbagg
    plot( idx, trg_acc, '--k')
    plot( idx, tst_acc, '-b')
    gca().set_ylim([0.8,1])

In [ ]:
# tensorflow mechanics
################################################################
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

NUM_CLASSES = 10 # The MNIST dataset has 10 classes, representing the digits 0 through 9.
IMAGE_SIZE = 28 # The MNIST images are always 28x28 pixels.
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE


# Basic model parameters 
flags = dict(
    learning_rate=0.01, # Initial learning rate
    max_steps=2000, #'Number of steps to run trainer
    hidden1=128, #Number of units in hidden layer 1.
    hidden2=2, #'Number of units in hidden layer 2.
    batch_size=100, #'Batch size.  Must divide evenly into the dataset sizes.
    train_dir='/home/ubuntu/fj/deep-learning/code/MNIST_data/', #Directory to put the training data
    fake_data=False, #If true, uses fake data for unit testing.
)

def placeholder_inputs(batch_size=None):
    # Note that the shapes of the placeholders match the shapes of the full image and label tensors, 
    # except that the first dimension is now batch_size
    images_placeholder = tf.placeholder( tf.float32, shape=(batch_size, IMAGE_PIXELS) )
    labels_placeholder = tf.placeholder( tf.int32, shape=(batch_size) )
    return images_placeholder, labels_placeholder

def inference(images, hidden1_units, hidden2_units):
    # Hidden layer 1
    with tf.name_scope('hidden1') as scope:
        weights = tf.Variable(
            tf.truncated_normal( [IMAGE_PIXELS, hidden1_units], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS)) ),
            name='weights' )
        biases = tf.Variable( tf.zeros([hidden1_units]), name='biases' )
        hidden1 = tf.nn.relu( tf.matmul(images, weights) + biases, name=scope )
    # Hidden layer 2
    with tf.name_scope('hidden2') as scope:
        weights = tf.Variable(
            tf.truncated_normal( [hidden1_units, hidden2_units], stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights' )
        biases = tf.Variable( tf.zeros([hidden2_units]), name='biases' )
        hidden2 = tf.nn.relu( tf.matmul(hidden1, weights) + biases, name=scope)
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal( [hidden2_units, NUM_CLASSES], stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable( tf.zeros([NUM_CLASSES]), name='biases' )
        logits = tf.matmul(hidden2, weights) + biases
    return logits #exp(logits_i)/sum_j exp(logits_j) = p_i

def lossfunction(logits, labels):
    labels = tf.to_int64( labels )
    #automatically produce 1-hot labels from the labels_placeholder and coverts logits into (log) probabilities.
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits( logits, labels, name='xentropy' )
    # average the cross entropy values across the batch dimension
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss

def training(loss, learning_rate):
    # tf.scalar_summary is an op for generating summary values into the events file when used with a SummaryWriter. 
    # it will emit the snapshot value of the loss every time summaries are written out.
    tf.scalar_summary( loss.op.name, loss )
    # Create the gradient descent optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Create a variable to track the global step. #note the trainable=False !!
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op

def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label."""
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1) of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))

def get_feed_dict(data_set, images_pl, labels_pl):
    images_feed, labels_feed = data_set.next_batch( flags['batch_size'], flags['fake_data'] )
    feed_dict = { images_pl: images_feed, labels_pl: tf.arg_max(labels_feed, 1).eval()}
    return feed_dict


def evaluate(sess, eval_correct, images_placeholder, labels_placeholder, data_set):
    """Evaluate against the full epoch of data. """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = data_set.num_examples // flags['batch_size']
    num_examples = steps_per_epoch * flags['batch_size']
    for step in range(steps_per_epoch):
        feed_dict = get_feed_dict(data_set, images_placeholder, labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = true_count / num_examples
    print( '  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision) )

# force gpu placement
with tf.device('/gpu:1'):
    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
        # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder = placeholder_inputs( flags['batch_size'] )
        # Build a Graph that computes predictions from the inference model.
        logits = inference(images_placeholder, flags['hidden1'], flags['hidden2'])
        # Add to the Graph the Ops for loss calculation.
        loss = lossfunction(logits, labels_placeholder)
        # Add to the Graph the Ops that calculate and apply gradients.
        train_op = training(loss, flags['learning_rate'])
        # Add the Op to compare the logits to the labels during evaluation.
        eval_correct = evaluation(logits, labels_placeholder)
        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.merge_all_summaries()
        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()
        # Create a session for running Ops on the Graph.    
        with tf.Session(  ) as sess:
            # Instantiate a SummaryWriter to output summaries and the Graph.
            summary_writer = tf.train.SummaryWriter( flags['train_dir'], sess.graph )
            # And then after everything is built, run the Op to initialize the variables.
            sess.run( tf.initialize_all_variables() )
            # Start the training loop.
            for step in range( flags['max_steps'] ):
                start_time = time.time()
                # Fill a feed dictionary with the actual set of images and labels for this particular training step.
                feed_dict = get_feed_dict(mnist.train, images_placeholder, labels_placeholder)
                # Run one step of the model.  The return values are the activations from the `train_op` 
                # (which is discarded) and the `loss` Op.  To inspect the values of your Ops or variables, 
                # you may inclutde them in the list passed to sess.run() and the value tensors will be
                # returned in the tuple from the call.
                _, loss_value = sess.run( [train_op, loss], feed_dict=feed_dict)
                duration = time.time() - start_time
                # Write the summaries and print an overview fairly often.
                if step % 100 == 0:
                    # Print status to stdout.
                    print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                    # Update the events file.
                    summary_str = sess.run( summary_op, feed_dict=feed_dict )
                    summary_writer.add_summary( summary_str, step )
                    summary_writer.flush()
                # Save a checkpoint and evaluate the model periodically.
                if (step + 1) % 1000 == 0 or (step + 1) == flags['max_steps']:
                    checkpoint_file = os.path.join( flags['train_dir'], 'checkpoint' )
                    saver.save( sess, checkpoint_file, global_step=step)
                    # Evaluate against the training set.
                    print('Training Data Eval:')
                    evaluate(sess, eval_correct, images_placeholder, labels_placeholder, mnist.train)
                    # Evaluate against the validation set.
                    print('Validation Data Eval:')
                    evaluate(sess, eval_correct, images_placeholder, labels_placeholder, mnist.validation)
                    # Evaluate against the test set.
                    print('Test Data Eval:')
                    evaluate(sess, eval_correct, images_placeholder, labels_placeholder, mnist.test)



In [ ]:
# tensorflow contrib logging tutorial
####################################################################################
import tensorflow as tf
import numpy as np

# Data sets
IRIS_TRAINING = "/home/ubuntu/fj/data/iris_training.csv"
IRIS_TEST = "/home/ubuntu/fj/data/iris_test.csv"

# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv(filename=IRIS_TRAINING,
                                                       target_dtype=np.int)
test_set = tf.contrib.learn.datasets.base.load_csv(filename=IRIS_TEST,
                                                   target_dtype=np.int)

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")

# Fit model.
classifier.fit(x=training_set.data,
               y=training_set.target,
               steps=2000)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(x=test_set.data,
                                     y=test_set.target)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
y = classifier.predict(new_samples)
print('Predictions: {}'.format(str(y)))

